In [2]:
import os
import requests
import aiohttp
import asyncio
import nest_asyncio
import pandas as pd
import time
import scrapy
from scrapy_playwright.page import PageMethod
from bs4 import BeautifulSoup
import nest_asyncio
import glob
import numpy as np
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz, process
import re


In [3]:
Fresh_Leads_formatted = pd.read_excel("Fresh_Leads_formatted.xlsx")
Fresh_Leads_formatted.shape

(368, 62)

In [4]:
# Filter Fresh_Leads_formatted to get rows where "PIC NAME 1 Contact Number" has a value (not NaN)
Fresh_Leads_Nov11 = Fresh_Leads_formatted[
    Fresh_Leads_formatted['PIC NAME 1 Contact Number'].notna()
].copy()

print(f"Original Fresh_Leads_formatted rows: {len(Fresh_Leads_formatted)}")
print(f"Filtered Fresh_Leads_Nov11 rows (with PIC NAME 1 Contact Number): {len(Fresh_Leads_Nov11)}")
print(f"Rows removed (NaN in PIC NAME 1 Contact Number): {len(Fresh_Leads_formatted) - len(Fresh_Leads_Nov11)}")

# Display the filtered dataframe
print("\n\nFresh_Leads_Nov11 (first 10 rows):")
Fresh_Leads_Nov11.head(10)


Original Fresh_Leads_formatted rows: 368
Filtered Fresh_Leads_Nov11 rows (with PIC NAME 1 Contact Number): 105
Rows removed (NaN in PIC NAME 1 Contact Number): 263


Fresh_Leads_Nov11 (first 10 rows):


,ePOS Code,Company Code,Date,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client,Tele Sales or MR (For KPI - Internal),Name of the Market Researcher,Original Source (Marketing),"Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)",...,PIC Name Designation 3,PIC NAME 3 Contact Number,PIC 3 email address,FB/Insta/Tik Tok/Linkedin Contact,Current ePOS Client ?,"If ePOS Client, which product they are using?",Is this deal part of the Gov List?,Source from Market Researcher,Contact Number from Lusha?,Phone number Verified ?
0,NaN,NaN,2025-11-10,BELL 1 SINGAPORE PTE. LTD.,BELL 1 SINGAPORE PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2025-11-10,THESEUS PROMOTER PTE. LTD.,THESEUS PROMOTER PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,2025-11-10,MYFIT ENTERPRISE PTE. LTD.,MYFIT ENTERPRISE PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,2025-11-10,THE SINGAPORE FOOD AND WINE,THE SINGAPORE FOOD AND WINE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,2025-11-10,SEA GUARDIAN MARINE (SINGAPORE) PTE. LTD.,SEA GUARDIAN MARINE (SINGAPORE) PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,NaN,2025-11-10,ABBACI DISTRIBUTION (SINGAPORE) PTE. LTD.,ABBACI DISTRIBUTION (SINGAPORE) PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,NaN,2025-11-10,PEM (SG) PTE. LTD.,PEM (SG) PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,NaN,NaN,2025-11-10,312 TECHNOLOGY LLP,312 TECHNOLOGY LLP,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaN,NaN,2025-11-10,SUSTAIN ENGINEERING PTE. LTD.,SUSTAIN ENGINEERING PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,2025-11-10,SLD SEMI (SINGAPORE) PTE. LTD.,SLD SEMI (SINGAPORE) PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
Fresh_Leads_Nov11

,ePOS Code,Company Code,Date,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client,Tele Sales or MR (For KPI - Internal),Name of the Market Researcher,Original Source (Marketing),"Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)",...,PIC Name Designation 3,PIC NAME 3 Contact Number,PIC 3 email address,FB/Insta/Tik Tok/Linkedin Contact,Current ePOS Client ?,"If ePOS Client, which product they are using?",Is this deal part of the Gov List?,Source from Market Researcher,Contact Number from Lusha?,Phone number Verified ?
0,NaN,NaN,2025-11-10,BELL 1 SINGAPORE PTE. LTD.,BELL 1 SINGAPORE PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2025-11-10,THESEUS PROMOTER PTE. LTD.,THESEUS PROMOTER PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,2025-11-10,MYFIT ENTERPRISE PTE. LTD.,MYFIT ENTERPRISE PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,2025-11-10,THE SINGAPORE FOOD AND WINE,THE SINGAPORE FOOD AND WINE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,2025-11-10,SEA GUARDIAN MARINE (SINGAPORE) PTE. LTD.,SEA GUARDIAN MARINE (SINGAPORE) PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,NaN,NaN,2025-11-10,ULTRA GREAT PTE. LTD.,ULTRA GREAT PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360,NaN,NaN,2025-11-10,SWEE HUAT TAN KEE TRADING,SWEE HUAT TAN KEE TRADING,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,NaN,NaN,2025-11-10,CC AVIATION SOLUTIONS PTE. LTD.,CC AVIATION SOLUTIONS PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,NaN,NaN,2025-11-10,A&K MARINE SUPPLIES PTE. LTD.,A&K MARINE SUPPLIES PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Remove [ ], ', +, and , (comma) from specified columns in Fresh_Leads_Nov11
columns_to_clean = [
    'PIC NAME 1 Contact Number',
    'PIC 1 email address',
    'Source from Market Researcher',
    'Business model',
    'Facebook Page',
    'Instagram URL',
    'Linkedin URL',
    'Tik Tok URL'
]

# Characters to remove: [ ] ' + ,
chars_to_remove = ['[', ']', "'", '+', ',']

# Clean each column by removing specified characters
for col in columns_to_clean:
    if col in Fresh_Leads_Nov11.columns:
        # Convert to string and remove all specified characters
        Fresh_Leads_Nov11[col] = Fresh_Leads_Nov11[col].astype(str)
        for char in chars_to_remove:
            Fresh_Leads_Nov11[col] = Fresh_Leads_Nov11[col].str.replace(char, '', regex=False)
        # Strip any extra whitespace
        Fresh_Leads_Nov11[col] = Fresh_Leads_Nov11[col].str.strip()
        # Replace 'nan' string back to actual NaN for empty values
        Fresh_Leads_Nov11[col] = Fresh_Leads_Nov11[col].replace('nan', pd.NA).replace('', pd.NA)
        print(f"Cleaned column: {col}")
    else:
        print(f"Warning: Column '{col}' not found in dataframe")

print(f"\nCharacters removed ([ ] ' + ,) from {len([c for c in columns_to_clean if c in Fresh_Leads_Nov11.columns])} columns")

# Function to format phone numbers to "65 XXXXXXXX" format with spaces
def format_phone_number(phone_str):
    """
    Format phone number to "65 XXXXXXXX" format with spaces.
    Extracts all digits and formats as: 65 [4 digits] [4 digits]
    """
    if pd.isna(phone_str) or phone_str == '' or phone_str == 'nan':
        return pd.NA
    
    # Extract all digits from the string
    digits_only = re.sub(r'\D', '', str(phone_str))
    
    # If empty after extracting digits, return NA
    if not digits_only:
        return pd.NA
    
    # Handle different formats
    # If starts with 65, format as "65 XXXX XXXX" (8 digits after 65)
    if digits_only.startswith('65') and len(digits_only) >= 10:
        # Remove the 65 prefix and get the remaining digits
        remaining = digits_only[2:]
        # Format as "65 XXXX XXXX" (4 digits, space, 4 digits)
        if len(remaining) >= 8:
            formatted = f"65 {remaining[:4]} {remaining[4:8]}"
            # If there are more digits, append them
            if len(remaining) > 8:
                formatted += f" {remaining[8:]}"
            return formatted
        else:
            # Less than 8 digits after 65, format as "65 XXXXXX"
            return f"65 {remaining}"
    
    # If doesn't start with 65 but has 8+ digits, assume it's a local number and add 65
    elif len(digits_only) >= 8:
        # Format as "65 XXXX XXXX"
        formatted = f"65 {digits_only[:4]} {digits_only[4:8]}"
        if len(digits_only) > 8:
            formatted += f" {digits_only[8:]}"
        return formatted
    
    # If less than 8 digits, just return as is (might be extension or incomplete)
    else:
        return digits_only

# Apply phone number formatting to "PIC NAME 1 Contact Number" column
if 'PIC NAME 1 Contact Number' in Fresh_Leads_Nov11.columns:
    Fresh_Leads_Nov11['PIC NAME 1 Contact Number'] = Fresh_Leads_Nov11['PIC NAME 1 Contact Number'].apply(format_phone_number)
    print("\nPhone numbers formatted to '65 XXXX XXXX' format")

print("\nSample of cleaned data:")
# Only show columns that exist in the dataframe
existing_columns = [col for col in columns_to_clean if col in Fresh_Leads_Nov11.columns]
if existing_columns:
    Fresh_Leads_Nov11[existing_columns].head(10)
else:
    print("No columns to display")


Cleaned column: PIC NAME 1 Contact Number
Cleaned column: PIC 1 email address
Cleaned column: Source from Market Researcher
Cleaned column: Business model
Cleaned column: Facebook Page
Cleaned column: Instagram URL
Cleaned column: Linkedin URL
Cleaned column: Tik Tok URL

Characters removed ([ ] ' + ,) from 8 columns

Phone numbers formatted to '65 XXXX XXXX' format

Sample of cleaned data:


In [20]:
Fresh_Leads_Nov11

,ePOS Code,Company Code,Date,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client,Tele Sales or MR (For KPI - Internal),Name of the Market Researcher,Original Source (Marketing),"Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)",...,PIC Name Designation 3,PIC NAME 3 Contact Number,PIC 3 email address,FB/Insta/Tik Tok/Linkedin Contact,Current ePOS Client ?,"If ePOS Client, which product they are using?",Is this deal part of the Gov List?,Source from Market Researcher,Contact Number from Lusha?,Phone number Verified ?
0,NaN,NaN,2025-11-10,BELL 1 SINGAPORE PTE. LTD.,BELL 1 SINGAPORE PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
5,NaN,NaN,2025-11-10,THESEUS PROMOTER PTE. LTD.,THESEUS PROMOTER PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
8,NaN,NaN,2025-11-10,MYFIT ENTERPRISE PTE. LTD.,MYFIT ENTERPRISE PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
13,NaN,NaN,2025-11-10,THE SINGAPORE FOOD AND WINE,THE SINGAPORE FOOD AND WINE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
17,NaN,NaN,2025-11-10,SEA GUARDIAN MARINE (SINGAPORE) PTE. LTD.,SEA GUARDIAN MARINE (SINGAPORE) PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,NaN,NaN,2025-11-10,ULTRA GREAT PTE. LTD.,ULTRA GREAT PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
360,NaN,NaN,2025-11-10,SWEE HUAT TAN KEE TRADING,SWEE HUAT TAN KEE TRADING,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
362,NaN,NaN,2025-11-10,CC AVIATION SOLUTIONS PTE. LTD.,CC AVIATION SOLUTIONS PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
364,NaN,NaN,2025-11-10,A&K MARINE SUPPLIES PTE. LTD.,A&K MARINE SUPPLIES PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN


In [21]:
Fresh_Leads_Nov11

,ePOS Code,Company Code,Date,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client,Tele Sales or MR (For KPI - Internal),Name of the Market Researcher,Original Source (Marketing),"Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)",...,PIC Name Designation 3,PIC NAME 3 Contact Number,PIC 3 email address,FB/Insta/Tik Tok/Linkedin Contact,Current ePOS Client ?,"If ePOS Client, which product they are using?",Is this deal part of the Gov List?,Source from Market Researcher,Contact Number from Lusha?,Phone number Verified ?
0,NaN,NaN,2025-11-10,BELL 1 SINGAPORE PTE. LTD.,BELL 1 SINGAPORE PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
5,NaN,NaN,2025-11-10,THESEUS PROMOTER PTE. LTD.,THESEUS PROMOTER PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
8,NaN,NaN,2025-11-10,MYFIT ENTERPRISE PTE. LTD.,MYFIT ENTERPRISE PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
13,NaN,NaN,2025-11-10,THE SINGAPORE FOOD AND WINE,THE SINGAPORE FOOD AND WINE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
17,NaN,NaN,2025-11-10,SEA GUARDIAN MARINE (SINGAPORE) PTE. LTD.,SEA GUARDIAN MARINE (SINGAPORE) PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,NaN,NaN,2025-11-10,ULTRA GREAT PTE. LTD.,ULTRA GREAT PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
360,NaN,NaN,2025-11-10,SWEE HUAT TAN KEE TRADING,SWEE HUAT TAN KEE TRADING,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
362,NaN,NaN,2025-11-10,CC AVIATION SOLUTIONS PTE. LTD.,CC AVIATION SOLUTIONS PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
364,NaN,NaN,2025-11-10,A&K MARINE SUPPLIES PTE. LTD.,A&K MARINE SUPPLIES PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN


In [22]:
# Check for duplicates in "PIC NAME 1 Contact Number" column
column_name = 'PIC NAME 1 Contact Number'

if column_name in Fresh_Leads_Nov11.columns:
    # Get the column data (excluding NaN values for duplicate check)
    phone_col = Fresh_Leads_Nov11[column_name].dropna()
    
    # Basic statistics
    total_rows = len(Fresh_Leads_Nov11)
    total_with_phone = len(phone_col)
    unique_phones = phone_col.nunique()
    duplicate_count = total_with_phone - unique_phones
    
    print("=" * 60)
    print("DUPLICATE ANALYSIS: PIC NAME 1 Contact Number")
    print("=" * 60)
    print(f"Total rows in dataframe: {total_rows}")
    print(f"Rows with phone number (non-NaN): {total_with_phone}")
    print(f"Unique phone numbers: {unique_phones}")
    print(f"Duplicate phone numbers: {duplicate_count}")
    print(f"Percentage of duplicates: {(duplicate_count/total_with_phone*100):.2f}%" if total_with_phone > 0 else "N/A")
    print()
    
    # Find duplicate phone numbers
    duplicate_phones = phone_col[phone_col.duplicated(keep=False)]
    
    if len(duplicate_phones) > 0:
        print("⚠️  DUPLICATES FOUND!")
        print(f"Number of rows with duplicate phone numbers: {len(duplicate_phones)}")
        print()
        
        # Count occurrences of each duplicate
        phone_counts = phone_col.value_counts()
        duplicates_only = phone_counts[phone_counts > 1]
        
        print(f"Phone numbers that appear more than once ({len(duplicates_only)} unique duplicates):")
        print("-" * 60)
        for phone, count in duplicates_only.items():
            print(f"  Phone: {phone} - appears {count} times")
        print()
        
        # Show all rows with duplicate phone numbers
        print("All rows with duplicate phone numbers:")
        print("-" * 60)
        duplicate_rows = Fresh_Leads_Nov11[Fresh_Leads_Nov11[column_name].isin(duplicates_only.index)]
        
        # Show key columns to identify the duplicates
        display_columns = [col for col in Fresh_Leads_Nov11.columns if any(keyword in col.lower() for keyword in ['company', 'name', 'phone', 'contact', 'email']) or col in ['PIC NAME 1 Contact Number']]
        if display_columns:
            display_df = duplicate_rows[display_columns].sort_values(by=column_name)
            print(f"\nShowing {len(display_df)} duplicate rows (sorted by phone number):")
            display(display_df)
        else:
            display(duplicate_rows.sort_values(by=column_name))
    else:
        print("✅ NO DUPLICATES FOUND!")
        print("All phone numbers in 'PIC NAME 1 Contact Number' are unique.")
else:
    print(f"❌ Error: Column '{column_name}' not found in dataframe")
    print(f"Available columns: {list(Fresh_Leads_Nov11.columns)}")


DUPLICATE ANALYSIS: PIC NAME 1 Contact Number
Total rows in dataframe: 105
Rows with phone number (non-NaN): 105
Unique phone numbers: 100
Duplicate phone numbers: 5
Percentage of duplicates: 4.76%

⚠️  DUPLICATES FOUND!
Number of rows with duplicate phone numbers: 6

Phone numbers that appear more than once (1 unique duplicates):
------------------------------------------------------------
  Phone: 65 9852 2583 - appears 6 times

All rows with duplicate phone numbers:
------------------------------------------------------------

Showing 6 duplicate rows (sorted by phone number):


,Company Code,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,Name of the Market Researcher,Company Registration date / Date Established,Company Registration Number (UEN),Hubspot ID (Company),Hubspot ID(Contact),First Name,Last Name,...,PIC NAME 2 Contact Number,PIC 2 email address,First Name 3,Last Name 3,PIC Name Designation 3,PIC NAME 3 Contact Number,PIC 3 email address,FB/Insta/Tik Tok/Linkedin Contact,Contact Number from Lusha?,Phone number Verified ?
291,NaN,OPTIMA CONTROL SYSTEMS (S.E.A.) PTE LTD,OPTIMA CONTROL SYSTEMS (S.E.A.) PTE LTD,NaN,17-08-2000,200007192D,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312,NaN,FORTRAN AVIATION PTE. LTD.,FORTRAN AVIATION PTE. LTD.,NaN,25-08-2008,200816801K,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329,NaN,AUSMAX PTE. LTD.,AUSMAX PTE. LTD.,NaN,13-10-2022,202236497W,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,NaN,RICHSEA ENTERPRISE PTE. LTD.,RICHSEA ENTERPRISE PTE. LTD.,NaN,21-12-2020,202041096W,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,NaN,CC AVIATION SOLUTIONS PTE. LTD.,CC AVIATION SOLUTIONS PTE. LTD.,NaN,09-12-2022,202243964H,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,NaN,A&K MARINE SUPPLIES PTE. LTD.,A&K MARINE SUPPLIES PTE. LTD.,NaN,06-04-2021,202112075G,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Remove duplicate phone numbers from "PIC NAME 1 Contact Number" column
column_name = 'PIC NAME 1 Contact Number'

if column_name in Fresh_Leads_Nov11.columns:
    # Get statistics before removal
    rows_before = len(Fresh_Leads_Nov11)
    phones_before = Fresh_Leads_Nov11[column_name].dropna().nunique()
    total_phones_before = Fresh_Leads_Nov11[column_name].notna().sum()
    
    print("=" * 60)
    print("REMOVING DUPLICATE PHONE NUMBERS")
    print("=" * 60)
    print(f"Rows before removal: {rows_before}")
    print(f"Unique phone numbers before: {phones_before}")
    print(f"Total rows with phone numbers: {total_phones_before}")
    print()
    
    # Remove duplicates, keeping the first occurrence
    # Separate rows with and without phone numbers
    rows_with_phone = Fresh_Leads_Nov11[Fresh_Leads_Nov11[column_name].notna()].copy()
    rows_without_phone = Fresh_Leads_Nov11[Fresh_Leads_Nov11[column_name].isna()].copy()
    
    # Remove duplicates from rows with phone numbers only
    rows_with_phone_deduplicated = rows_with_phone.drop_duplicates(
        subset=[column_name], 
        keep='first'
    ).copy()
    
    # Combine back: deduplicated phone rows + all rows without phone (they're not duplicates)
    Fresh_Leads_Nov11_deduplicated = pd.concat(
        [rows_with_phone_deduplicated, rows_without_phone], 
        ignore_index=True
    )
    
    # Get statistics after removal
    rows_after = len(Fresh_Leads_Nov11_deduplicated)
    phones_after = Fresh_Leads_Nov11_deduplicated[column_name].dropna().nunique()
    total_phones_after = Fresh_Leads_Nov11_deduplicated[column_name].notna().sum()
    rows_removed = rows_before - rows_after
    
    print("Results after duplicate removal:")
    print(f"Rows after removal: {rows_after}")
    print(f"Unique phone numbers after: {phones_after}")
    print(f"Total rows with phone numbers: {total_phones_after}")
    print(f"Rows removed: {rows_removed}")
    print(f"Percentage reduction: {(rows_removed/rows_before*100):.2f}%")
    print()
    
    # Update the original dataframe
    Fresh_Leads_Nov11 = Fresh_Leads_Nov11_deduplicated.copy()
    
    print("✅ Duplicate phone numbers removed successfully!")
    print(f"Fresh_Leads_Nov11 now contains {len(Fresh_Leads_Nov11)} unique records.")
    print()
    print("First 10 rows of deduplicated dataframe:")
    print("-" * 60)
    display(Fresh_Leads_Nov11.head(10))
else:
    print(f"❌ Error: Column '{column_name}' not found in dataframe")
    print(f"Available columns: {list(Fresh_Leads_Nov11.columns)}")


REMOVING DUPLICATE PHONE NUMBERS
Rows before removal: 105
Unique phone numbers before: 100
Total rows with phone numbers: 105

Results after duplicate removal:
Rows after removal: 100
Unique phone numbers after: 100
Total rows with phone numbers: 100
Rows removed: 5
Percentage reduction: 4.76%

✅ Duplicate phone numbers removed successfully!
Fresh_Leads_Nov11 now contains 100 unique records.

First 10 rows of deduplicated dataframe:
------------------------------------------------------------


,ePOS Code,Company Code,Date,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client,Tele Sales or MR (For KPI - Internal),Name of the Market Researcher,Original Source (Marketing),"Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)",...,PIC Name Designation 3,PIC NAME 3 Contact Number,PIC 3 email address,FB/Insta/Tik Tok/Linkedin Contact,Current ePOS Client ?,"If ePOS Client, which product they are using?",Is this deal part of the Gov List?,Source from Market Researcher,Contact Number from Lusha?,Phone number Verified ?
0,NaN,NaN,2025-11-10,BELL 1 SINGAPORE PTE. LTD.,BELL 1 SINGAPORE PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2025-11-10,THESEUS PROMOTER PTE. LTD.,THESEUS PROMOTER PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2025-11-10,MYFIT ENTERPRISE PTE. LTD.,MYFIT ENTERPRISE PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,2025-11-10,THE SINGAPORE FOOD AND WINE,THE SINGAPORE FOOD AND WINE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,2025-11-10,SEA GUARDIAN MARINE (SINGAPORE) PTE. LTD.,SEA GUARDIAN MARINE (SINGAPORE) PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2025-11-10,ABBACI DISTRIBUTION (SINGAPORE) PTE. LTD.,ABBACI DISTRIBUTION (SINGAPORE) PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,2025-11-10,PEM (SG) PTE. LTD.,PEM (SG) PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,2025-11-10,312 TECHNOLOGY LLP,312 TECHNOLOGY LLP,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,2025-11-10,SUSTAIN ENGINEERING PTE. LTD.,SUSTAIN ENGINEERING PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,2025-11-10,SLD SEMI (SINGAPORE) PTE. LTD.,SLD SEMI (SINGAPORE) PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
print(Fresh_Leads_Nov11[Fresh_Leads_Nov11['PIC NAME 1 Contact Number'].duplicated(keep=False)])


Empty DataFrame
Columns: [ePOS Code, Company Code, Date, ACRA REGISTERED NAME, Brand/Deal Name/Business Name, Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client, Tele Sales or MR (For KPI - Internal), Name of the Market Researcher, Original Source (Marketing), Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients), Company Registration date / Date Established, Company Registration Number (UEN), Primary SSIC Code, Secondary SSIC Code, Hubspot ID (Company), Hubspot ID(Deal), Hubspot ID(Contact), Website URL, Business Type, Facebook Page, Instagram URL, Linkedin URL, Tik Tok URL, Ownership Type, Parent Industry Type, Industry Type, Sub Industry, Business model, Presence of Multiple Outlets, Number of Outlets (Write in #), Region, Planning Area, Business Location Type, Registered Address (Block & Street), Registered Address  (Unit #), Registered Address  (Postal code), Operational Address 
(Block & Street), Operational Address 

In [25]:
# Remove duplicate Website URLs - keep first occurrence, set duplicates to NaN
if 'Website URL' in Fresh_Leads_formatted.columns:
    # Count duplicates before removal
    total_rows = len(Fresh_Leads_formatted)
    valid_urls = Fresh_Leads_formatted['Website URL'].notna()
    unique_urls_before = Fresh_Leads_formatted[valid_urls]['Website URL'].nunique()
    duplicate_count = Fresh_Leads_formatted[valid_urls]['Website URL'].duplicated().sum()
    
    print(f"Before removing duplicates:")
    print(f"  Total rows: {total_rows}")
    print(f"  Rows with Website URL: {valid_urls.sum()}")
    print(f"  Unique URLs: {unique_urls_before}")
    print(f"  Duplicate URLs: {duplicate_count}")
    
    # Mark duplicates (keep first occurrence, mark subsequent as duplicates)
    # Convert to string and normalize (lowercase, strip whitespace) for comparison
    Fresh_Leads_formatted['Website URL_cleaned'] = Fresh_Leads_formatted['Website URL'].astype(str).str.strip().str.lower()
    
    # Find duplicates (keep first occurrence)
    is_duplicate = Fresh_Leads_formatted['Website URL_cleaned'].duplicated(keep='first')
    
    # Set duplicate URLs to NaN (excluding rows where URL was already NaN)
    mask_to_remove = is_duplicate & (Fresh_Leads_formatted['Website URL'].notna())
    Fresh_Leads_formatted.loc[mask_to_remove, 'Website URL'] = pd.NA
    
    # Drop the temporary cleaning column
    Fresh_Leads_formatted = Fresh_Leads_formatted.drop(columns=['Website URL_cleaned'])
    
    # Count after removal
    valid_urls_after = Fresh_Leads_formatted['Website URL'].notna()
    unique_urls_after = Fresh_Leads_formatted[valid_urls_after]['Website URL'].nunique()
    
    print(f"\nAfter removing duplicates:")
    print(f"  Rows with Website URL: {valid_urls_after.sum()}")
    print(f"  Unique URLs: {unique_urls_after}")
    print(f"  Duplicates removed: {duplicate_count}")
    print(f"  Rows with duplicate URLs set to NaN: {mask_to_remove.sum()}")
else:
    print("Warning: 'Website URL' column not found in dataframe")


Before removing duplicates:
  Total rows: 368
  Rows with Website URL: 121
  Unique URLs: 116
  Duplicate URLs: 5

After removing duplicates:
  Rows with Website URL: 116
  Unique URLs: 116
  Duplicates removed: 5
  Rows with duplicate URLs set to NaN: 5


In [26]:
# Remove duplicate Facebook Page, Instagram URL, and PIC 1 email address - keep first occurrence, set duplicates to NaN
columns_to_clean = ['Facebook Page', 'Instagram URL', 'PIC 1 email address']

for col_name in columns_to_clean:
    if col_name in Fresh_Leads_formatted.columns:
        print(f"\n{'='*60}")
        print(f"Processing: {col_name}")
        print(f"{'='*60}")
        
        # Count duplicates before removal
        valid_values = Fresh_Leads_formatted[col_name].notna()
        unique_before = Fresh_Leads_formatted[valid_values][col_name].nunique()
        duplicate_count = Fresh_Leads_formatted[valid_values][col_name].duplicated().sum()
        
        print(f"Before removing duplicates:")
        print(f"  Rows with {col_name}: {valid_values.sum()}")
        print(f"  Unique values: {unique_before}")
        print(f"  Duplicate values: {duplicate_count}")
        
        # Normalize values for comparison (lowercase, strip whitespace)
        temp_col = f'{col_name}_cleaned'
        Fresh_Leads_formatted[temp_col] = Fresh_Leads_formatted[col_name].astype(str).str.strip().str.lower()
        
        # Find duplicates (keep first occurrence)
        is_duplicate = Fresh_Leads_formatted[temp_col].duplicated(keep='first')
        
        # Set duplicate values to NaN (excluding rows where value was already NaN)
        mask_to_remove = is_duplicate & (Fresh_Leads_formatted[col_name].notna())
        Fresh_Leads_formatted.loc[mask_to_remove, col_name] = pd.NA
        
        # Drop the temporary cleaning column
        Fresh_Leads_formatted = Fresh_Leads_formatted.drop(columns=[temp_col])
        
        # Count after removal
        valid_after = Fresh_Leads_formatted[col_name].notna()
        unique_after = Fresh_Leads_formatted[valid_after][col_name].nunique()
        
        print(f"\nAfter removing duplicates:")
        print(f"  Rows with {col_name}: {valid_after.sum()}")
        print(f"  Unique values: {unique_after}")
        print(f"  Duplicates removed: {mask_to_remove.sum()}")
        print(f"  ✅ Successfully removed {mask_to_remove.sum()} duplicate values")
    else:
        print(f"⚠️  Warning: '{col_name}' column not found in dataframe")

print(f"\n{'='*60}")
print("Duplicate removal complete for Facebook Page, Instagram URL, and PIC 1 email address!")
print(f"{'='*60}")



Processing: Facebook Page
Before removing duplicates:
  Rows with Facebook Page: 57
  Unique values: 52
  Duplicate values: 5

After removing duplicates:
  Rows with Facebook Page: 52
  Unique values: 52
  Duplicates removed: 5
  ✅ Successfully removed 5 duplicate values

Processing: Instagram URL
Before removing duplicates:
  Rows with Instagram URL: 36
  Unique values: 36
  Duplicate values: 0

After removing duplicates:
  Rows with Instagram URL: 36
  Unique values: 36
  Duplicates removed: 0
  ✅ Successfully removed 0 duplicate values

Processing: PIC 1 email address
Before removing duplicates:
  Rows with PIC 1 email address: 66
  Unique values: 61
  Duplicate values: 5

After removing duplicates:
  Rows with PIC 1 email address: 61
  Unique values: 61
  Duplicates removed: 5
  ✅ Successfully removed 5 duplicate values

Duplicate removal complete for Facebook Page, Instagram URL, and PIC 1 email address!


In [27]:
# AGGRESSIVE DATE REMOVAL for Operational Address (Unit #) - Remove ALL dates!
# Find the column
op_col = None
for col in Fresh_Leads_formatted.columns:
    if 'operational' in col.lower() and 'unit' in col.lower() and 'address' in col.lower():
        op_col = col
        break

if op_col:
    print(f"AGGRESSIVELY cleaning: '{op_col}'\n")
    s = Fresh_Leads_formatted[op_col].astype(str)
    to_remove = []
    month_names = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'sept', 'oct', 'nov', 'dec']
    
    for idx, val in s.items():
        if pd.isna(val) or str(val).strip().lower() in ['nan', 'none', '', 'nat']:
            continue
        val_str = str(val).strip()
        val_lower = val_str.lower()
        
        # KEEP only if it has a letter (02A, 12W, 330G) - these are unit numbers
        if re.search(r'[A-Za-z]', val_str):
            continue  # Has letter = unit number, KEEP IT
        
        # No letters - check if it's a date and REMOVE
        is_date = False
        
        # Check for month names
        for month in month_names:
            if month in val_lower:
                is_date = True
                break
        
        # Check date patterns
        if re.search(r'\d{4}[-/]\d{1,2}[-/]\d{1,2}', val_str):
            is_date = True
        elif re.search(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}', val_str):
            is_date = True
        elif re.match(r'^\d{1,2}[-/]\d{1,2}$', val_str):
            # Could be MM/DD - check if ranges suggest date
            parts = re.split(r'[-/]', val_str)
            if len(parts) == 2:
                try:
                    p1, p2 = int(parts[0]), int(parts[1])
                    if (1 <= p1 <= 31) and (1 <= p2 <= 12):
                        is_date = True
                except:
                    pass
        
        # Try pandas date parsing
        if not is_date:
            try:
                pd.to_datetime(val_str, errors='raise')
                is_date = True
            except:
                pass
        
        if is_date:
            to_remove.append(idx)
    
    # Remove all dates
    print(f"Removing {len(to_remove)} date values...")
    if to_remove:
        print("Examples of dates being removed:")
        for idx in to_remove[:15]:
            print(f"  Row {idx}: {s.loc[idx]}")
        if len(to_remove) > 15:
            print(f"  ... and {len(to_remove) - 15} more")
    
    Fresh_Leads_formatted.loc[to_remove, op_col] = pd.NA
    
    print(f"\n✅ Done! Removed {len(to_remove)} dates.")
    print(f"Remaining non-null values: {Fresh_Leads_formatted[op_col].notna().sum()}")
    print("\nSample of remaining values:")
    print(Fresh_Leads_formatted[Fresh_Leads_formatted[op_col].notna()][[op_col]].head(15))
else:
    print("Column not found!")


AGGRESSIVELY cleaning: 'Operational Address 
(Unit #)'

Removing 0 date values...

✅ Done! Removed 0 dates.
Remaining non-null values: 0

Sample of remaining values:
Empty DataFrame
Columns: [Operational Address 
(Unit #)]
Index: []


In [28]:
# Check and format all date columns to ddmmyy format
print("=" * 60)
print("CHECKING AND FORMATTING DATE COLUMNS")
print("=" * 60)

# Find all date-related columns
date_keywords = ['date', 'Date', 'DATE', 'established', 'Established', 'registration', 'Registration']
date_columns = [col for col in Fresh_Leads_Nov11.columns if any(keyword in col for keyword in date_keywords)]

print(f"\nFound {len(date_columns)} potential date columns:")
for col in date_columns:
    print(f"  - {col}")

print("\n" + "-" * 60)

# Function to format date to ddmmyy
def format_date_to_ddmmyy(date_value):
    """
    Convert date to ddmmyy format.
    Handles various input formats and converts to DD-MM-YY format.
    """
    if pd.isna(date_value) or date_value == '' or str(date_value).strip().lower() in ['nan', 'none', 'nat']:
        return pd.NA
    
    try:
        # Try to parse the date using pandas
        date_parsed = pd.to_datetime(date_value, errors='coerce', dayfirst=True)
        
        if pd.isna(date_parsed):
            # If parsing fails, return original value
            return date_value
        
        # Format as DD-MM-YY
        formatted_date = date_parsed.strftime('%d-%m-%y')
        return formatted_date
    except:
        # If any error occurs, return original value
        return date_value

# Process each date column
formatted_count = 0
for col in date_columns:
    print(f"\nProcessing column: '{col}'")
    
    # Count non-null values before
    non_null_before = Fresh_Leads_Nov11[col].notna().sum()
    print(f"  Non-null values: {non_null_before}")
    
    if non_null_before > 0:
        # Show sample of current values
        sample_values = Fresh_Leads_Nov11[Fresh_Leads_Nov11[col].notna()][col].head(5)
        print(f"  Sample values before formatting:")
        for idx, val in sample_values.items():
            print(f"    Row {idx}: {val}")
        
        # Apply formatting
        Fresh_Leads_Nov11[col] = Fresh_Leads_Nov11[col].apply(format_date_to_ddmmyy)
        formatted_count += 1
        
        # Show sample of formatted values
        sample_formatted = Fresh_Leads_Nov11[Fresh_Leads_Nov11[col].notna()][col].head(5)
        print(f"  Sample values after formatting:")
        for idx, val in sample_formatted.items():
            print(f"    Row {idx}: {val}")
    else:
        print(f"  No values to format (all NaN)")

print("\n" + "=" * 60)
print(f"✅ Date formatting complete! Formatted {formatted_count} date columns.")
print("=" * 60)


CHECKING AND FORMATTING DATE COLUMNS

Found 3 potential date columns:
  - Date
  - Company Registration date / Date Established
  - Company Registration Number (UEN)

------------------------------------------------------------

Processing column: 'Date'
  Non-null values: 100
  Sample values before formatting:
    Row 0: 2025-11-10 00:00:00
    Row 1: 2025-11-10 00:00:00
    Row 2: 2025-11-10 00:00:00
    Row 3: 2025-11-10 00:00:00
    Row 4: 2025-11-10 00:00:00
  Sample values after formatting:
    Row 0: 10-11-25
    Row 1: 10-11-25
    Row 2: 10-11-25
    Row 3: 10-11-25
    Row 4: 10-11-25

Processing column: 'Company Registration date / Date Established'
  Non-null values: 100
  Sample values before formatting:
    Row 0: 07-06-2018
    Row 1: 07-05-2019
    Row 2: 01-03-2022
    Row 3: 03-07-2020
    Row 4: 09-07-2019
  Sample values after formatting:
    Row 0: 07-06-18
    Row 1: 07-05-19
    Row 2: 01-03-22
    Row 3: 03-07-20
    Row 4: 09-07-19

Processing column: 'Company 

In [29]:
# Double-check for duplicates in PIC NAME 1 Contact Number and remove if found
print("=" * 60)
print("FINAL DUPLICATE CHECK: PIC NAME 1 Contact Number")
print("=" * 60)

column_name = 'PIC NAME 1 Contact Number'

if column_name in Fresh_Leads_Nov11.columns:
    # Get statistics
    total_rows = len(Fresh_Leads_Nov11)
    phone_col = Fresh_Leads_Nov11[column_name].dropna()
    total_with_phone = len(phone_col)
    unique_phones = phone_col.nunique()
    duplicate_count = total_with_phone - unique_phones
    
    print(f"\nCurrent Status:")
    print(f"  Total rows: {total_rows}")
    print(f"  Rows with phone number: {total_with_phone}")
    print(f"  Unique phone numbers: {unique_phones}")
    print(f"  Duplicate phone numbers: {duplicate_count}")
    
    # Check for duplicates
    if duplicate_count > 0:
        print(f"\n⚠️  DUPLICATES FOUND! Removing duplicates...")
        
        # Count occurrences of each duplicate
        phone_counts = phone_col.value_counts()
        duplicates_only = phone_counts[phone_counts > 1]
        
        print(f"\nPhone numbers that appear more than once ({len(duplicates_only)} unique duplicates):")
        for phone, count in duplicates_only.items():
            print(f"  Phone: {phone} - appears {count} times")
        
        # Remove duplicates, keeping the first occurrence
        rows_before = len(Fresh_Leads_Nov11)
        
        # Separate rows with and without phone numbers
        rows_with_phone = Fresh_Leads_Nov11[Fresh_Leads_Nov11[column_name].notna()].copy()
        rows_without_phone = Fresh_Leads_Nov11[Fresh_Leads_Nov11[column_name].isna()].copy()
        
        # Remove duplicates from rows with phone numbers only
        rows_with_phone_deduplicated = rows_with_phone.drop_duplicates(
            subset=[column_name], 
            keep='first'
        ).copy()
        
        # Combine back
        Fresh_Leads_Nov11 = pd.concat(
            [rows_with_phone_deduplicated, rows_without_phone], 
            ignore_index=True
        )
        
        rows_after = len(Fresh_Leads_Nov11)
        rows_removed = rows_before - rows_after
        
        # Verify no duplicates remain
        phone_col_after = Fresh_Leads_Nov11[column_name].dropna()
        unique_phones_after = phone_col_after.nunique()
        duplicate_count_after = len(phone_col_after) - unique_phones_after
        
        print(f"\nAfter removing duplicates:")
        print(f"  Rows removed: {rows_removed}")
        print(f"  Total rows: {rows_after}")
        print(f"  Unique phone numbers: {unique_phones_after}")
        print(f"  Remaining duplicates: {duplicate_count_after}")
        
        if duplicate_count_after == 0:
            print(f"\n✅ SUCCESS! All duplicates removed. No duplicates remaining.")
        else:
            print(f"\n⚠️  WARNING: {duplicate_count_after} duplicates still remain. Please check manually.")
    else:
        print(f"\n✅ NO DUPLICATES FOUND! All phone numbers are unique.")
        print(f"   No action needed.")
    
    print("\n" + "=" * 60)
    print("Final verification complete!")
    print("=" * 60)
else:
    print(f"❌ Error: Column '{column_name}' not found in dataframe")
    print(f"Available columns: {list(Fresh_Leads_Nov11.columns)}")


FINAL DUPLICATE CHECK: PIC NAME 1 Contact Number

Current Status:
  Total rows: 100
  Rows with phone number: 100
  Unique phone numbers: 100
  Duplicate phone numbers: 0

✅ NO DUPLICATES FOUND! All phone numbers are unique.
   No action needed.

Final verification complete!


In [30]:
# Clean "Brand/Deal Name/Business Name" column - Remove PTE, LTD, and all variants
print("=" * 60)
print("CLEANING BRAND/DEAL NAME/BUSINESS NAME COLUMN")
print("=" * 60)

column_name = 'Brand/Deal Name/Business Name'

if column_name in Fresh_Leads_Nov11.columns:
    # Count non-null values before
    non_null_before = Fresh_Leads_Nov11[column_name].notna().sum()
    print(f"\nNon-null values before cleaning: {non_null_before}")
    
    if non_null_before > 0:
        # Show sample of current values
        print("\nSample values BEFORE cleaning:")
        sample_before = Fresh_Leads_Nov11[Fresh_Leads_Nov11[column_name].notna()][column_name].head(10)
        for idx, val in sample_before.items():
            print(f"  Row {idx}: {val}")
        
        def clean_business_name(name):
            """
            Remove legal entity suffixes (PTE, LTD, etc.) from business name.
            Handles various formats and combinations.
            """
            if pd.isna(name) or name == '' or str(name).strip().lower() in ['nan', 'none']:
                return pd.NA
            
            # Convert to string and strip whitespace
            cleaned = str(name).strip()
            original = cleaned
            
            # Remove patterns in order (most specific first)
            # Pattern 1: (SG) PTE LTD / (SINGAPORE) PTE LTD variations
            cleaned = re.sub(r'\s*\(SG\)\s*PTE\s*\.?\s*LTD\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r'\s*\(SINGAPORE\)\s*PTE\s*\.?\s*LTD\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r'\s*SINGAPORE\s+PTE\s*\.?\s*LTD\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            
            # Pattern 2: PTE LTD / LTD PTE combinations (with or without dots)
            cleaned = re.sub(r'\s+PTE\s*\.?\s*LTD\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r'\s+LTD\s*\.?\s*PTE\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r'\s+PTE\s+LTD\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r'\s+LTD\s+PTE\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            
            # Pattern 3: Remove (SG) or (SINGAPORE) at the end if they remain
            cleaned = re.sub(r'\s*\(SG\)\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r'\s*\(SINGAPORE\)\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            
            # Pattern 4: Single entity types
            cleaned = re.sub(r'\s+PTE\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r'\s+LTD\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r'\s+LLP\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r'\s+LLC\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r'\s+INC\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r'\s+CORP\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r'\s+CORPORATION\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r'\s+CO\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r'\s+COMPANY\s*\.?\s*$', '', cleaned, flags=re.IGNORECASE)
            
            # Clean up extra spaces and trailing punctuation
            cleaned = re.sub(r'\s+', ' ', cleaned)  # Multiple spaces to single space
            cleaned = cleaned.strip()  # Remove leading/trailing whitespace
            cleaned = re.sub(r'\s*[,\-\.]+\s*$', '', cleaned)  # Remove trailing commas, dashes, dots
            cleaned = cleaned.strip()  # Strip again after removing punctuation
            
            # If empty after cleaning, return original
            if not cleaned or cleaned == '':
                return original
            
            return cleaned
        
        # Apply cleaning
        Fresh_Leads_Nov11[column_name] = Fresh_Leads_Nov11[column_name].apply(clean_business_name)
        
        # Show sample of cleaned values
        print("\nSample values AFTER cleaning:")
        sample_after = Fresh_Leads_Nov11[Fresh_Leads_Nov11[column_name].notna()][column_name].head(10)
        for idx, val in sample_after.items():
            print(f"  Row {idx}: {val}")
        
        # Count changes
        changes_count = 0
        for idx in Fresh_Leads_Nov11.index:
            if Fresh_Leads_Nov11[column_name].notna().loc[idx]:
                original = str(sample_before.loc[idx]) if idx in sample_before.index else None
                cleaned = str(Fresh_Leads_Nov11[column_name].loc[idx])
                if original and original != cleaned:
                    changes_count += 1
        
        print(f"\n✅ Cleaning complete!")
        print(f"   Processed {non_null_before} non-null values")
        print(f"   Business names cleaned and legal entity suffixes removed")
    else:
        print(f"\n⚠️  No values to clean (all NaN)")
    
    print("\n" + "=" * 60)
else:
    print(f"❌ Error: Column '{column_name}' not found in dataframe")
    print(f"Available columns: {list(Fresh_Leads_Nov11.columns)}")


CLEANING BRAND/DEAL NAME/BUSINESS NAME COLUMN

Non-null values before cleaning: 100

Sample values BEFORE cleaning:
  Row 0: BELL 1 SINGAPORE PTE. LTD.
  Row 1: THESEUS PROMOTER PTE. LTD.
  Row 2: MYFIT ENTERPRISE PTE. LTD.
  Row 3: THE SINGAPORE FOOD AND WINE
  Row 4: SEA GUARDIAN MARINE (SINGAPORE) PTE. LTD.
  Row 5: ABBACI DISTRIBUTION (SINGAPORE) PTE. LTD.
  Row 6: PEM (SG) PTE. LTD.
  Row 7: 312 TECHNOLOGY LLP
  Row 8: SUSTAIN ENGINEERING PTE. LTD.
  Row 9: SLD SEMI (SINGAPORE) PTE. LTD.

Sample values AFTER cleaning:
  Row 0: BELL 1
  Row 1: THESEUS PROMOTER
  Row 2: MYFIT ENTERPRISE
  Row 3: THE SINGAPORE FOOD AND WINE
  Row 4: SEA GUARDIAN MARINE
  Row 5: ABBACI DISTRIBUTION
  Row 6: PEM
  Row 7: 312 TECHNOLOGY
  Row 8: SUSTAIN ENGINEERING
  Row 9: SLD SEMI

✅ Cleaning complete!
   Processed 100 non-null values
   Business names cleaned and legal entity suffixes removed



In [32]:
Fresh_Leads_Nov11.to_excel("Fresh_Leads_Nov11.xlsx", index=False)
